In [9]:
import numpy as np
import astropy
import matplotlib
import astropy.units as u
from astropy.coordinates import AltAz, EarthLocation, SkyCoord
from astropy.time import Time
from datetime import datetime

from ugradio import leusch

import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [12, 10]

! potential azimuth issue: ~5.2

In [8]:
datetime.now()

datetime.datetime(2023, 4, 17, 23, 31, 20, 927994)

In [44]:
# leusch
LOC = EarthLocation(lat=37.91934*u.deg, lon=122.15385*u.deg, height=304*u.m)
ALT_MIN, ALT_MAX = leusch.ALT_MIN, leusch.ALT_MAX
AZ_MIN, AZ_MAX = leusch.AZ_MIN, leusch.AZ_MAX

def galactic_to_altaz(coord, time):
    l, b = coord
    g_sc = SkyCoord(frame="galactic", l=l*u.deg, b=b*u.deg)
    altaz_sc = g_sc.transform_to(AltAz(obstime=time, location=LOC))
    return altaz_sc.alt.degree, altaz_sc.az.degree

def observable_time(coord, time):
    '''
    given:
    target (source) in galactic coordinates [LAT, LON]
    time as an astropy time object
    
    assume:
    location of telescope 
    range of telescope
    
    return:
    a "countdown time" for the target coordinate 
    that counts the **quantity of observable altitude degrees**
    until it is no longer in observable range
    '''
    alt, az = galactic_to_altaz(coord, time)
    if (ALT_MIN < alt < ALT_MAX) and (AZ_MIN < az < AZ_MAX):
        if 0 <= az < 180:
            p = (ALT_MAX - alt) + (ALT_MAX - ALT_MIN)
        else:
            p = alt - ALT_MIN
    else:
        p = 0
    assert p >= 0, 'bad p'
    return p
    

In [46]:
galactic_to_altaz([120,0], Time(datetime.now()))

(59.50760863518871, 25.841796442598437)

In [45]:
observable_time([120,0], Time(datetime.now()))

95.49471437982214